In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 54 kB 1.5 MB/s eta 0:00:011
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-py3-none-any.whl size=73198 sha256=c6116546b18cdf1798a04686691a8689f65297f8a33f811d5e854c5b73dfaf17
  Stored in directory: /root/.cache/pip/wheels/0b/cf/2f/1a1749d3a3650fac3305a8d7f9237b6de7c41068e2f8520ca2
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=ff7562a3aee993e36585ced8a963860c59ba038c2f65978f258fc306ca6884e8
  Stored in directory: /root/.cache/pip/wheels/ba/ad/c8/2d98360791161cd3db6daf6b5e730f34021fc9367d5879f497
Successfully built keras-tuner terminaltables
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()


4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [12]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - ETA: 1:46:58 - loss: 2.2660 - accuracy: 0.093 - ETA: 9:48 - loss: 1.9602 - accuracy: 0.3665  - ETA: 6:02 - loss: 1.6199 - accuracy: 0.44 - ETA: 4:23 - loss: 1.4789 - accuracy: 0.49 - ETA: 3:09 - loss: 1.3042 - accuracy: 0.55 - ETA: 2:35 - loss: 1.2000 - accuracy: 0.58 - ETA: 2:02 - loss: 1.1161 - accuracy: 0.60 - ETA: 1:42 - loss: 1.0446 - accuracy: 0.62 - ETA: 1:26 - loss: 0.9897 - accuracy: 0.65 - ETA: 1:13 - loss: 0.9403 - accuracy: 0.66 - ETA: 1:03 - loss: 0.9026 - accuracy: 0.67 - ETA: 56s - loss: 0.8704 - accuracy: 0.6860 - ETA: 50s - loss: 0.8457 - accuracy: 0.695 - ETA: 46s - loss: 0.8163 - accuracy: 0.704 - ETA: 43s - loss: 0.7986 - accuracy: 0.711 - ETA: 40s - loss: 0.7785 - accuracy: 0.719 - ETA: 37s - loss: 0.7665 - accuracy: 0.723 - ETA: 35s - loss: 0.7496 - accuracy: 0.729 - ETA: 33s - loss: 0.7396 - accuracy: 0.732 - ETA: 31s - loss: 0.7282 - accuracy: 0.737 - ETA: 2

54000/54000 [==============================] - ETA: 9s - loss: 0.2749 - accuracy: 0.93 - ETA: 6s - loss: 0.2388 - accuracy: 0.90 - ETA: 6s - loss: 0.2329 - accuracy: 0.91 - ETA: 6s - loss: 0.2171 - accuracy: 0.91 - ETA: 6s - loss: 0.2209 - accuracy: 0.91 - ETA: 6s - loss: 0.2127 - accuracy: 0.92 - ETA: 6s - loss: 0.2094 - accuracy: 0.92 - ETA: 6s - loss: 0.2138 - accuracy: 0.92 - ETA: 6s - loss: 0.2122 - accuracy: 0.92 - ETA: 6s - loss: 0.2163 - accuracy: 0.92 - ETA: 6s - loss: 0.2166 - accuracy: 0.92 - ETA: 6s - loss: 0.2187 - accuracy: 0.92 - ETA: 6s - loss: 0.2201 - accuracy: 0.92 - ETA: 6s - loss: 0.2193 - accuracy: 0.92 - ETA: 6s - loss: 0.2163 - accuracy: 0.92 - ETA: 6s - loss: 0.2149 - accuracy: 0.92 - ETA: 6s - loss: 0.2109 - accuracy: 0.92 - ETA: 6s - loss: 0.2124 - accuracy: 0.92 - ETA: 5s - loss: 0.2138 - accuracy: 0.92 - ETA: 5s - loss: 0.2154 - accuracy: 0.92 - ETA: 5s - loss: 0.2125 - accuracy: 0.92 - ETA: 5s - loss: 0.2126 - accuracy: 0.92 - ETA: 5s - loss: 0.2159 - accu

Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - ETA: 10:05 - loss: 2.3087 - accuracy: 0.093 - ETA: 57s - loss: 1.5907 - accuracy: 0.4766  - ETA: 33s - loss: 1.3179 - accuracy: 0.558 - ETA: 25s - loss: 1.1619 - accuracy: 0.615 - ETA: 21s - loss: 1.0514 - accuracy: 0.648 - ETA: 18s - loss: 0.9672 - accuracy: 0.675 - ETA: 16s - loss: 0.9048 - accuracy: 0.691 - ETA: 15s - loss: 0.8620 - accuracy: 0.701 - ETA: 14s - loss: 0.8321 - accuracy: 0.708 - ETA: 13s - loss: 0.8018 - accuracy: 0.718 - ETA: 12s - loss: 0.7754 - accuracy: 0.727 - ETA: 12s - loss: 0.7558 - accuracy: 0.734 - ETA: 11s - loss: 0.7394 - accuracy: 0.741 - ETA: 11s - loss: 0.7181 - accuracy: 0.747 - ETA: 11s - loss: 0.7018 - accuracy: 0.752 - ETA: 10s - loss: 0.6820 - accuracy: 0.759 - ETA: 10s - loss: 0.6691 - accuracy: 0.762 - ETA: 10s - loss: 0.6566 - accuracy: 0.767 - ETA: 10s - loss: 0.6471 - accuracy: 0.771 - ETA: 9s - loss: 0.6341 - accuracy: 0.776 - ETA: 9s - l

54000/54000 [==============================] - ETA: 12s - loss: 0.1490 - accuracy: 0.937 - ETA: 7s - loss: 0.1688 - accuracy: 0.951 - ETA: 7s - loss: 0.1591 - accuracy: 0.94 - ETA: 7s - loss: 0.1533 - accuracy: 0.94 - ETA: 7s - loss: 0.1572 - accuracy: 0.94 - ETA: 7s - loss: 0.1534 - accuracy: 0.94 - ETA: 7s - loss: 0.1524 - accuracy: 0.94 - ETA: 7s - loss: 0.1600 - accuracy: 0.94 - ETA: 7s - loss: 0.1635 - accuracy: 0.93 - ETA: 6s - loss: 0.1593 - accuracy: 0.94 - ETA: 6s - loss: 0.1587 - accuracy: 0.94 - ETA: 6s - loss: 0.1555 - accuracy: 0.94 - ETA: 6s - loss: 0.1535 - accuracy: 0.94 - ETA: 6s - loss: 0.1584 - accuracy: 0.94 - ETA: 6s - loss: 0.1586 - accuracy: 0.94 - ETA: 6s - loss: 0.1576 - accuracy: 0.94 - ETA: 6s - loss: 0.1572 - accuracy: 0.94 - ETA: 6s - loss: 0.1574 - accuracy: 0.94 - ETA: 6s - loss: 0.1549 - accuracy: 0.94 - ETA: 6s - loss: 0.1559 - accuracy: 0.94 - ETA: 6s - loss: 0.1558 - accuracy: 0.94 - ETA: 6s - loss: 0.1561 - accuracy: 0.94 - ETA: 6s - loss: 0.1538 - a

Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - ETA: 10:00 - loss: 2.3089 - accuracy: 0.031 - ETA: 52s - loss: 1.6615 - accuracy: 0.3942  - ETA: 30s - loss: 1.3248 - accuracy: 0.523 - ETA: 22s - loss: 1.1192 - accuracy: 0.598 - ETA: 19s - loss: 1.0207 - accuracy: 0.632 - ETA: 17s - loss: 0.9413 - accuracy: 0.657 - ETA: 15s - loss: 0.8909 - accuracy: 0.672 - ETA: 14s - loss: 0.8338 - accuracy: 0.690 - ETA: 13s - loss: 0.8178 - accuracy: 0.695 - ETA: 12s - loss: 0.7940 - accuracy: 0.706 - ETA: 11s - loss: 0.7729 - accuracy: 0.715 - ETA: 11s - loss: 0.7547 - accuracy: 0.723 - ETA: 10s - loss: 0.7324 - accuracy: 0.731 - ETA: 10s - loss: 0.7170 - accuracy: 0.737 - ETA: 10s - loss: 0.7061 - accuracy: 0.741 - ETA: 9s - loss: 0.6939 - accuracy: 0.747 - ETA: 9s - loss: 0.6877 - accuracy: 0.75 - ETA: 9s - loss: 0.6720 - accuracy: 0.75 - ETA: 9s - loss: 0.6640 - accuracy: 0.75 - ETA: 8s - loss: 0.6544 - accuracy: 0.76 - ETA: 8s - loss: 0.6

54000/54000 [==============================] - ETA: 12s - loss: 0.2693 - accuracy: 0.968 - ETA: 8s - loss: 0.1981 - accuracy: 0.919 - ETA: 7s - loss: 0.1711 - accuracy: 0.93 - ETA: 7s - loss: 0.1581 - accuracy: 0.93 - ETA: 7s - loss: 0.1652 - accuracy: 0.93 - ETA: 7s - loss: 0.1591 - accuracy: 0.93 - ETA: 7s - loss: 0.1560 - accuracy: 0.93 - ETA: 7s - loss: 0.1546 - accuracy: 0.93 - ETA: 8s - loss: 0.1607 - accuracy: 0.93 - ETA: 8s - loss: 0.1610 - accuracy: 0.93 - ETA: 7s - loss: 0.1586 - accuracy: 0.93 - ETA: 7s - loss: 0.1661 - accuracy: 0.93 - ETA: 7s - loss: 0.1675 - accuracy: 0.93 - ETA: 7s - loss: 0.1688 - accuracy: 0.93 - ETA: 7s - loss: 0.1686 - accuracy: 0.93 - ETA: 7s - loss: 0.1714 - accuracy: 0.93 - ETA: 7s - loss: 0.1743 - accuracy: 0.93 - ETA: 7s - loss: 0.1729 - accuracy: 0.93 - ETA: 7s - loss: 0.1721 - accuracy: 0.93 - ETA: 7s - loss: 0.1703 - accuracy: 0.93 - ETA: 7s - loss: 0.1695 - accuracy: 0.93 - ETA: 7s - loss: 0.1689 - accuracy: 0.93 - ETA: 7s - loss: 0.1690 - a

Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - ETA: 9:22 - loss: 2.3221 - accuracy: 0.06 - ETA: 49s - loss: 2.4438 - accuracy: 0.1466 - ETA: 29s - loss: 2.0861 - accuracy: 0.273 - ETA: 21s - loss: 1.8153 - accuracy: 0.373 - ETA: 17s - loss: 1.6132 - accuracy: 0.440 - ETA: 15s - loss: 1.4530 - accuracy: 0.496 - ETA: 13s - loss: 1.3468 - accuracy: 0.531 - ETA: 13s - loss: 1.2741 - accuracy: 0.557 - ETA: 13s - loss: 1.2378 - accuracy: 0.567 - ETA: 12s - loss: 1.1652 - accuracy: 0.591 - ETA: 11s - loss: 1.1123 - accuracy: 0.605 - ETA: 10s - loss: 1.0690 - accuracy: 0.621 - ETA: 10s - loss: 1.0317 - accuracy: 0.631 - ETA: 10s - loss: 0.9982 - accuracy: 0.644 - ETA: 10s - loss: 0.9803 - accuracy: 0.650 - ETA: 10s - loss: 0.9628 - accuracy: 0.656 - ETA: 10s - loss: 0.9399 - accuracy: 0.664 - ETA: 9s - loss: 0.9256 - accuracy: 0.670 - ETA: 9s - loss: 0.9062 - accuracy: 0.67 - ETA: 9s - loss: 0.8948 - accuracy: 0.68 - ETA: 9s - loss: 0.

54000/54000 [==============================] - ETA: 11s - loss: 0.4499 - accuracy: 0.812 - ETA: 7s - loss: 0.3212 - accuracy: 0.891 - ETA: 8s - loss: 0.3297 - accuracy: 0.87 - ETA: 9s - loss: 0.3499 - accuracy: 0.86 - ETA: 8s - loss: 0.3654 - accuracy: 0.86 - ETA: 8s - loss: 0.3690 - accuracy: 0.86 - ETA: 7s - loss: 0.3696 - accuracy: 0.86 - ETA: 7s - loss: 0.3607 - accuracy: 0.86 - ETA: 7s - loss: 0.3539 - accuracy: 0.86 - ETA: 7s - loss: 0.3537 - accuracy: 0.86 - ETA: 7s - loss: 0.3588 - accuracy: 0.86 - ETA: 7s - loss: 0.3524 - accuracy: 0.86 - ETA: 7s - loss: 0.3507 - accuracy: 0.86 - ETA: 7s - loss: 0.3536 - accuracy: 0.86 - ETA: 7s - loss: 0.3592 - accuracy: 0.86 - ETA: 6s - loss: 0.3598 - accuracy: 0.86 - ETA: 6s - loss: 0.3562 - accuracy: 0.86 - ETA: 6s - loss: 0.3594 - accuracy: 0.86 - ETA: 6s - loss: 0.3608 - accuracy: 0.86 - ETA: 6s - loss: 0.3639 - accuracy: 0.86 - ETA: 6s - loss: 0.3644 - accuracy: 0.86 - ETA: 6s - loss: 0.3625 - accuracy: 0.86 - ETA: 6s - loss: 0.3615 - a

Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - ETA: 10:20 - loss: 2.2930 - accuracy: 0.125 - ETA: 49s - loss: 3.2955 - accuracy: 0.2210  - ETA: 28s - loss: 2.3538 - accuracy: 0.391 - ETA: 20s - loss: 1.9110 - accuracy: 0.485 - ETA: 17s - loss: 1.6605 - accuracy: 0.529 - ETA: 14s - loss: 1.4851 - accuracy: 0.568 - ETA: 13s - loss: 1.3651 - accuracy: 0.588 - ETA: 12s - loss: 1.2745 - accuracy: 0.607 - ETA: 11s - loss: 1.2096 - accuracy: 0.622 - ETA: 10s - loss: 1.1496 - accuracy: 0.634 - ETA: 10s - loss: 1.1014 - accuracy: 0.649 - ETA: 9s - loss: 1.0612 - accuracy: 0.659 - ETA: 9s - loss: 1.0301 - accuracy: 0.66 - ETA: 8s - loss: 1.0019 - accuracy: 0.67 - ETA: 8s - loss: 0.9768 - accuracy: 0.67 - ETA: 8s - loss: 0.9489 - accuracy: 0.68 - ETA: 8s - loss: 0.9314 - accuracy: 0.68 - ETA: 7s - loss: 0.9103 - accuracy: 0.69 - ETA: 7s - loss: 0.8953 - accuracy: 0.69 - ETA: 7s - loss: 0.8791 - accuracy: 0.70 - ETA: 7s - loss: 0.8610 - ac

54000/54000 [==============================] - ETA: 12s - loss: 0.2922 - accuracy: 0.843 - ETA: 7s - loss: 0.3996 - accuracy: 0.861 - ETA: 6s - loss: 0.3915 - accuracy: 0.85 - ETA: 6s - loss: 0.3764 - accuracy: 0.86 - ETA: 6s - loss: 0.3851 - accuracy: 0.85 - ETA: 6s - loss: 0.3661 - accuracy: 0.86 - ETA: 6s - loss: 0.3601 - accuracy: 0.86 - ETA: 6s - loss: 0.3617 - accuracy: 0.86 - ETA: 6s - loss: 0.3708 - accuracy: 0.86 - ETA: 6s - loss: 0.3752 - accuracy: 0.85 - ETA: 6s - loss: 0.3757 - accuracy: 0.85 - ETA: 6s - loss: 0.3729 - accuracy: 0.86 - ETA: 6s - loss: 0.3723 - accuracy: 0.86 - ETA: 5s - loss: 0.3738 - accuracy: 0.86 - ETA: 5s - loss: 0.3746 - accuracy: 0.86 - ETA: 5s - loss: 0.3699 - accuracy: 0.86 - ETA: 5s - loss: 0.3699 - accuracy: 0.86 - ETA: 5s - loss: 0.3665 - accuracy: 0.86 - ETA: 5s - loss: 0.3671 - accuracy: 0.86 - ETA: 5s - loss: 0.3707 - accuracy: 0.86 - ETA: 5s - loss: 0.3712 - accuracy: 0.86 - ETA: 5s - loss: 0.3750 - accuracy: 0.86 - ETA: 5s - loss: 0.3777 - a

In [14]:
model=tuner_search.get_best_models(num_models=1)[0]

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        41520     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3539072   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 3,582,842
Trainable params: 3,582,842
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Train on 54000 samples, validate on 6000 samples
Epoch 4/10
54000/54000 [==============================] - ETA: 8:46 - loss: 0.0542 - accuracy: 1.00 - ETA: 50s - loss: 0.0742 - accuracy: 0.9740 - ETA: 29s - loss: 0.0789 - accuracy: 0.971 - ETA: 21s - loss: 0.0787 - accuracy: 0.972 - ETA: 18s - loss: 0.0881 - accuracy: 0.970 - ETA: 16s - loss: 0.0903 - accuracy: 0.970 - ETA: 14s - loss: 0.0916 - accuracy: 0.968 - ETA: 13s - loss: 0.0919 - accuracy: 0.967 - ETA: 12s - loss: 0.0926 - accuracy: 0.966 - ETA: 11s - loss: 0.0942 - accuracy: 0.965 - ETA: 11s - loss: 0.0935 - accuracy: 0.965 - ETA: 10s - loss: 0.0972 - accuracy: 0.964 - ETA: 10s - loss: 0.0961 - accuracy: 0.963 - ETA: 10s - loss: 0.0957 - accuracy: 0.964 - ETA: 9s - loss: 0.0957 - accuracy: 0.964 - ETA: 9s - loss: 0.0966 - accuracy: 0.96 - ETA: 9s - loss: 0.0972 - accuracy: 0.96 - ETA: 9s - loss: 0.0964 - accuracy: 0.96 - ETA: 9s - loss: 0.0967 - accuracy: 0.96 - ETA: 8s - loss: 0.0966 - accuracy: 0.96 - ETA: 8s - loss: 0.0974 

54000/54000 [==============================] - ETA: 9s - loss: 0.0391 - accuracy: 0.96 - ETA: 7s - loss: 0.0380 - accuracy: 0.99 - ETA: 7s - loss: 0.0353 - accuracy: 0.99 - ETA: 7s - loss: 0.0343 - accuracy: 0.99 - ETA: 7s - loss: 0.0373 - accuracy: 0.98 - ETA: 7s - loss: 0.0355 - accuracy: 0.99 - ETA: 7s - loss: 0.0343 - accuracy: 0.99 - ETA: 7s - loss: 0.0351 - accuracy: 0.99 - ETA: 7s - loss: 0.0359 - accuracy: 0.98 - ETA: 7s - loss: 0.0354 - accuracy: 0.98 - ETA: 7s - loss: 0.0365 - accuracy: 0.98 - ETA: 7s - loss: 0.0364 - accuracy: 0.98 - ETA: 7s - loss: 0.0361 - accuracy: 0.98 - ETA: 7s - loss: 0.0364 - accuracy: 0.98 - ETA: 7s - loss: 0.0370 - accuracy: 0.98 - ETA: 7s - loss: 0.0377 - accuracy: 0.98 - ETA: 7s - loss: 0.0370 - accuracy: 0.98 - ETA: 7s - loss: 0.0371 - accuracy: 0.98 - ETA: 7s - loss: 0.0374 - accuracy: 0.98 - ETA: 7s - loss: 0.0366 - accuracy: 0.98 - ETA: 6s - loss: 0.0365 - accuracy: 0.98 - ETA: 6s - loss: 0.0359 - accuracy: 0.98 - ETA: 6s - loss: 0.0354 - accu

54000/54000 [==============================] - ETA: 8s - loss: 0.0220 - accuracy: 1.00 - ETA: 7s - loss: 0.0215 - accuracy: 0.99 - ETA: 7s - loss: 0.0248 - accuracy: 0.99 - ETA: 7s - loss: 0.0193 - accuracy: 0.99 - ETA: 7s - loss: 0.0217 - accuracy: 0.99 - ETA: 7s - loss: 0.0223 - accuracy: 0.99 - ETA: 7s - loss: 0.0232 - accuracy: 0.99 - ETA: 7s - loss: 0.0220 - accuracy: 0.99 - ETA: 7s - loss: 0.0222 - accuracy: 0.99 - ETA: 7s - loss: 0.0218 - accuracy: 0.99 - ETA: 7s - loss: 0.0234 - accuracy: 0.99 - ETA: 7s - loss: 0.0227 - accuracy: 0.99 - ETA: 7s - loss: 0.0225 - accuracy: 0.99 - ETA: 7s - loss: 0.0223 - accuracy: 0.99 - ETA: 7s - loss: 0.0216 - accuracy: 0.99 - ETA: 6s - loss: 0.0230 - accuracy: 0.99 - ETA: 6s - loss: 0.0239 - accuracy: 0.99 - ETA: 6s - loss: 0.0231 - accuracy: 0.99 - ETA: 6s - loss: 0.0224 - accuracy: 0.99 - ETA: 6s - loss: 0.0225 - accuracy: 0.99 - ETA: 6s - loss: 0.0225 - accuracy: 0.99 - ETA: 6s - loss: 0.0224 - accuracy: 0.99 - ETA: 6s - loss: 0.0223 - accu

54000/54000 [==============================] - ETA: 14s - loss: 0.0397 - accuracy: 0.968 - ETA: 7s - loss: 0.0194 - accuracy: 0.992 - ETA: 7s - loss: 0.0135 - accuracy: 0.99 - ETA: 7s - loss: 0.0117 - accuracy: 0.99 - ETA: 7s - loss: 0.0109 - accuracy: 0.99 - ETA: 7s - loss: 0.0115 - accuracy: 0.99 - ETA: 7s - loss: 0.0106 - accuracy: 0.99 - ETA: 7s - loss: 0.0099 - accuracy: 0.99 - ETA: 7s - loss: 0.0100 - accuracy: 0.99 - ETA: 7s - loss: 0.0126 - accuracy: 0.99 - ETA: 7s - loss: 0.0119 - accuracy: 0.99 - ETA: 7s - loss: 0.0131 - accuracy: 0.99 - ETA: 6s - loss: 0.0129 - accuracy: 0.99 - ETA: 6s - loss: 0.0121 - accuracy: 0.99 - ETA: 6s - loss: 0.0116 - accuracy: 0.99 - ETA: 6s - loss: 0.0111 - accuracy: 0.99 - ETA: 6s - loss: 0.0108 - accuracy: 0.99 - ETA: 6s - loss: 0.0105 - accuracy: 0.99 - ETA: 6s - loss: 0.0100 - accuracy: 0.99 - ETA: 6s - loss: 0.0109 - accuracy: 0.99 - ETA: 6s - loss: 0.0108 - accuracy: 0.99 - ETA: 6s - loss: 0.0110 - accuracy: 0.99 - ETA: 6s - loss: 0.0109 - a